# Sentiment prediction using Tensorflow 2.0

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import pandas as pd
import csv
import numpy as np
from tokenizer_class import FullTokenizer, is_number
import multiprocessing as mp
import pickle
from sklearn import preprocessing
from sklearn import metrics

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import backend as K
from tensorflow.keras.models import load_model

pd.set_option('max_colwidth', 100)
%load_ext autoreload
%autoreload 2

### Word Embeddings
Load the pre-trained GloVe 300 dimensional embedding matrix
Obtained here: https://nlp.stanford.edu/projects/glove/

In [2]:
embedding_file = 'data/glove.6B.300d.txt'

df_embedding = pd.read_csv(embedding_file, sep=' ', index_col=0,
                           header=None, na_values=None,
                           keep_default_na=False,
                           quoting=csv.QUOTE_NONE)
df_embedding.sort_index(inplace=True)

In [3]:
df_embedding.shape

(400000, 300)

In [4]:
df_embedding.sample(10)

,1,2,3,4,5,6,7,8,9,10,...,291,292,293,294,295,296,297,298,299,300
0,,,,,,,,,,,,,,,,,,,,,
sonar,0.33048,0.116850,-0.411260,0.188480,0.321090,-0.813960,-0.051148,-0.337450,-0.204780,-1.198300,...,-0.858400,0.252970,-0.539270,-0.595040,-0.248270,-1.113000,0.42782,-0.236000,0.375960,-0.427000
hearted,-0.20698,-0.404830,-0.267510,-0.591330,-0.064752,0.043845,0.084845,-0.105650,0.011034,-0.076297,...,-0.041760,0.238870,-0.110440,0.238910,0.282370,-0.652840,-0.58310,0.072921,-0.067318,0.518940
dosage,-0.65587,0.827610,-0.055756,0.357710,0.394540,-0.125710,-0.314500,-0.197430,-0.250880,-0.710290,...,-0.332710,0.171840,0.229170,-0.506400,0.223940,-0.711440,-0.51024,-0.298890,-0.090484,-0.322730
lightyears,-0.22943,-0.682830,-0.280610,0.480470,-0.135190,-0.379870,-0.172250,-0.115170,0.552260,0.839840,...,0.395030,0.395180,0.264700,-0.000818,0.009533,-0.878330,-0.37824,0.267460,-0.076411,-0.030604
ragni,-0.13872,0.161070,0.525340,0.042151,-0.436300,-0.484830,0.134350,0.033304,0.360310,0.768940,...,0.460640,0.338860,0.124590,-0.131970,-0.106400,-0.601260,0.25891,1.098600,0.432760,-0.302610
poshteh-ye,-0.56235,0.084654,-0.066012,0.231860,-0.147000,-0.511670,0.572680,0.350820,-0.723060,1.059200,...,0.047565,0.557360,0.006690,0.371090,0.268260,-0.045595,-0.53871,0.076358,0.279560,-0.607250
mdivani,0.36266,-0.045557,-0.034230,0.157770,-0.653320,-0.478830,0.234880,-0.129390,0.160240,0.981830,...,-0.099246,0.012156,0.445630,0.029854,0.237290,-0.553470,0.21815,0.682050,0.038624,-0.342590
harriers,0.38212,0.665550,0.366030,0.118270,0.145510,0.104760,0.538450,0.458350,0.065454,0.874270,...,-1.532600,0.975060,-0.196270,-0.611260,-0.785580,-0.455280,0.37727,-0.640130,-0.900120,0.579210
kolah,-0.25497,0.078224,-0.314990,-0.011261,0.041182,-0.240370,-0.041194,0.519860,-0.778400,0.574290,...,-0.341470,0.626570,0.012646,-0.449710,0.166700,-1.003600,-0.15172,0.197940,-0.014791,0.423450


#### Cosine Similarity
$\Large \text{sim(U, V)} = \frac {U^T\cdot{V}} {||U||_2 ||V||_2} = cos(\theta)$

<img src="img/cosine_sim.png">

In [5]:
def cosine_similarity(u, v):
    cs = np.dot(u, v) / (np.linalg.norm(u, ord=2) * np.linalg.norm(v, ord=2))
    return cs

In [6]:
cosine_similarity(df_embedding.loc['france'], df_embedding.loc['italy'])

0.5643459466769057

In [7]:
cosine_similarity(df_embedding.loc['ball'], df_embedding.loc['crocodile'])

0.025038104793071447

In [8]:
cosine_similarity(df_embedding.loc['france'] - df_embedding.loc['paris'],
                  df_embedding.loc['rome'] - df_embedding.loc['italy'])


-0.6228158930510513

In [9]:
with open('data/google-10000-english.txt', 'r') as f:
    vocab = f.read().splitlines()
df_embedding_small = df_embedding.reindex(vocab)

In [10]:
def word_analogy(a, b, c):
    emb_a, emb_b, emb_c = df_embedding_small.loc[a], \
                          df_embedding_small.loc[b], \
                          df_embedding_small.loc[c]
    max_sim = -100
    result = None
    for w, emb_w in df_embedding_small.iterrows():
        if w in [a, b, c]:
            continue
        sim = cosine_similarity(emb_b - emb_a, emb_w - emb_c)
        if sim > max_sim:
            max_sim = sim
            result = w
    return result

In [11]:
test_cases = [
    ('paris', 'france', 'london'),
    ('paris', 'france', 'rome'),
    ('paris', 'france', 'madrid'),
    ('paris', 'france', 'berlin'),
    ('man', 'king', 'woman'),
    ('walk', 'walked', 'go'),
    ('man', 'woman', 'boy'),
    ('apple', 'juice', 'mother')
]

for t in test_cases:
    r = word_analogy(t[0], t[1], t[2])
    print('{} -> {} : {} -> {}'.format(t[0], t[1], t[2], r))

paris -> france : london -> britain
paris -> france : rome -> italy
paris -> france : madrid -> spain
paris -> france : berlin -> germany
man -> king : woman -> queen
walk -> walked : go -> went
man -> woman : boy -> girl
apple -> juice : mother -> lemon


#### Load data

In [12]:
news_data = pd.read_csv('data/news_data.csv',
                        usecols = ['TEXT', 'SENTIMENT'])
news_data.drop_duplicates(inplace=True)

In [13]:
news_data.tail()

,SENTIMENT,TEXT
611430,0.94,Commonwealth Bank of Australia Upgraded to Buy from Hold by Bell Potter
611431,-0.58,Qbe Insurance Downgraded to Hold from Buy by Bell Potter
611432,0.49,Collective & Roland Team for
611433,-0.58,Mcmillan Shakespeare Downgraded to Neutral from Outperform by Macquarie
611434,0.95,Nearmap Initiated at Outperform by Macquarie


#### Initialize the tokenizer and compress the embedding matrix

Build out a set of all tokens from every `TEXT` record using the full tokenizer (with its vocab initialised from the embedding matrix), then filter the embedding matrix dictionary to only include those tokens that exist in the `TEXT`. Then, re-instantiate the tokenizer using the embedding matrix dictionary. This speeds up training for testing/POC purpose by minimizing the size of the embedding tensor.

In [14]:
# load saved tokenizer?
if True:
    with open('model/tokenizer.pkl', 'rb') as f:
        tokenizer = pickle.load(f)
    df_embedding = df_embedding.reindex(tokenizer.vocab.keys())
    df_embedding.sort_index(inplace=True)
    df_embedding.loc['<START>'] = np.zeros(df_embedding.shape[1])
    df_embedding.loc['<UNK>'] = df_embedding.mean()
    df_embedding.loc['<NUMBER>'] = np.ones(df_embedding.shape[1])/2
    df_embedding.loc['<PAD>'] = np.ones(df_embedding.shape[1])
    
else:
    tokenizer = FullTokenizer(df_embedding.index.values)

    def process(arr):
        for i in map(tokenizer.tokenize, arr):
            tokens.update(i)
        return tokens

    tokens = set()
    if __name__ == '__main__':
        p = mp.Pool(processes=mp.cpu_count())
        split_arr = np.array_split(news_data.TEXT.values, mp.cpu_count())
        pool_results = p.map(process, split_arr)
        p.close()
        p.join()

        all_tokens = set.union(*pool_results)
    
    all_tokens = set([t for t in all_tokens if not is_number(t)])
    df_embedding = df_embedding.reindex(all_tokens).dropna()
    df_embedding.sort_index(inplace=True)
    df_embedding.loc['<START>'] = np.zeros(df_embedding.shape[1])
    df_embedding.loc['<UNK>'] = df_embedding.mean()
    df_embedding.loc['<NUMBER>'] = np.ones(df_embedding.shape[1])/2
    df_embedding.loc['<PAD>'] = np.ones(df_embedding.shape[1])

tokenizer = FullTokenizer(df_embedding.index.values)

Tokenization example

In [15]:
test_sentence = 'Tokenizer takes a string and breaks it down into tokens.'
tokenizer.tokenize(test_sentence)

['tokenizer',
 'takes',
 'a',
 'string',
 'and',
 'breaks',
 'it',
 'down',
 'into',
 'tokens',
 '.']

In [16]:
tokenizer.convert_tokens_to_ids(tokenizer.tokenize(test_sentence))

[1241, 56549, 1252, 55330, 3529, 8641, 29211, 17049, 28807, 58228, 19]

Visualise tokens in the word embeddings matrix

In [17]:
df_embedding.iloc[tokenizer.convert_tokens_to_ids(tokenizer.tokenize(test_sentence))]

,1,2,3,4,5,6,7,8,9,10,...,291,292,293,294,295,296,297,298,299,300
0,,,,,,,,,,,,,,,,,,,,,
<UNK>,0.056789,-0.022786,-0.002645,0.069256,0.010156,-0.036345,0.032409,-0.064922,-0.041482,0.234542,...,0.020075,0.102891,0.090614,-0.024519,-0.070710,-0.191057,0.048234,0.136562,0.088450,-0.031001
takes,-0.265480,-0.109470,0.179890,-0.077156,-0.001871,-0.027737,-0.180900,-0.145750,0.341820,-1.314900,...,-0.278790,0.128100,-0.213730,-0.148400,-0.123810,0.011745,-0.331660,0.319620,-0.323520,0.002213
a,-0.297120,0.094049,-0.096662,-0.344000,-0.184830,-0.123290,-0.116560,-0.099692,0.172650,-1.638600,...,0.075972,-0.424260,-0.396700,0.326830,0.620490,0.347190,0.269520,0.059717,-0.228530,0.296020
string,0.490790,0.373550,-0.108000,-0.181940,-0.406680,-0.214930,-0.200550,-0.469240,-0.120310,-0.924760,...,0.161770,-0.332490,-0.319580,0.347060,0.116960,0.870680,0.264700,-0.543940,-0.368430,-0.225760
and,0.038466,-0.039792,0.082747,-0.389230,-0.214310,0.170200,-0.025657,0.095780,0.238600,-1.634200,...,0.045194,-0.204050,-0.210970,-0.110250,0.021766,0.441290,0.327970,-0.334270,0.011807,0.059703
breaks,-0.273560,0.012609,-0.441310,0.410770,0.134850,0.144790,0.190910,-0.257050,0.442460,-0.934800,...,-0.304630,0.051168,-0.794090,0.129680,-0.291050,-0.219350,-0.609280,-0.054624,0.389920,0.752370
it,0.033284,-0.040754,-0.048377,0.120170,-0.139150,-0.176940,-0.062908,0.170560,0.200770,-2.428700,...,0.091222,-0.402000,0.154300,0.230990,0.086138,-0.002428,0.065196,-0.154080,0.178060,-0.196830
down,-0.081429,-0.110040,-0.031034,0.604570,0.067606,-0.316090,-0.460590,-0.202730,0.468520,-1.600900,...,0.212930,-0.172900,0.013380,0.450640,0.124720,0.549970,-0.196470,-0.390020,-0.062974,0.096852
into,-0.182340,-0.192090,0.151670,-0.048763,0.283540,-0.025568,-0.398820,-0.014072,0.410290,-1.987000,...,-0.058528,-0.133670,0.159020,0.072976,0.779430,0.792400,0.073351,-0.248390,0.144680,0.176830


#### Define encoding/decoding methods
The `encode` method takes a string and converts it to IDs using the tokenizer dictionary. `decode` performs the opposite action.

In [18]:
def encode(string):
    encoding = ["<START>"] + tokenizer.tokenize(string)
    encoding = tokenizer.convert_tokens_to_ids(encoding)
    return encoding

def decode(ids):
    tokens = tokenizer.convert_ids_to_tokens(ids)
    return tokens

#### Process the dataset
We encode `TEXT` using the tokenizer to obtain IDs that correspond to each token's index within the embedding matrix. We use multiprocessing to process large datasets.

In [19]:
def process(df):
    res = df.TEXT.apply(encode)
    return res

if __name__ == '__main__':
    p = mp.Pool(processes=mp.cpu_count())
    split_dfs = np.array_split(news_data, mp.cpu_count())
    pool_results = p.map(process, split_dfs)
    p.close()
    p.join()

    # merging parts processed by different processes
    dataset = pd.concat(pool_results, axis=0)

Preview the dataset:

In [20]:
dataset.tail()

611430    [1240, 12768, 5885, 41528, 5144, 60208, 58166, 9550, 21961, 26614, 9589, 6649, 45148]
611431                [1240, 46376, 28611, 17057, 58166, 26614, 21961, 9550, 9589, 6649, 45148]
611432                                                    [1240, 12567, 5, 49267, 56981, 21423]
611433                     [1240, 36652, 52076, 17057, 58166, 40196, 21961, 42449, 9589, 35053]
611434                                            [1240, 1241, 28416, 4837, 42449, 9589, 35053]
Name: TEXT, dtype: object

Let's decode the sample:

In [21]:
dataset.tail().apply(decode)

611430    [<START>, commonwealth, bank, of, australia, upgraded, to, buy, from, hold, by, bell, potter]
611431                     [<START>, qbe, insurance, downgraded, to, hold, from, buy, by, bell, potter]
611432                                                      [<START>, collective, &, roland, team, for]
611433       [<START>, mcmillan, shakespeare, downgraded, to, neutral, from, outperform, by, macquarie]
611434                                       [<START>, <UNK>, initiated, at, outperform, by, macquarie]
Name: TEXT, dtype: object

#### Pad all sequences to maxlen (inferred from the dataset) to obtain equal length vectors

In [22]:
maxlen = dataset.apply(len).max() + 10
data = keras.preprocessing.sequence.pad_sequences(dataset.values,
                                           value=tokenizer.vocab["<PAD>"],
                                           padding='post',
                                           maxlen=maxlen)

#### Build the model

<img src="img/lstm_diagram.png">

In [26]:
neurons = 128
dropout = 0.40

my_embedding = layers.Embedding(input_dim=df_embedding.shape[0],
                                output_dim=df_embedding.shape[1],
                                input_length=maxlen,
                                trainable=False)

my_embedding.build((None, ))
my_embedding.set_weights([df_embedding.values])

model = keras.Sequential([
    my_embedding,
    layers.LSTM(neurons, return_sequences=True),
    layers.Dropout(dropout),
    layers.LSTM(neurons, return_sequences=False),
    layers.Dropout(dropout),
    layers.Dense(1, activation='tanh')
])

optimizer = tf.optimizers.Adam(learning_rate=0.001)

def r2_keras(y_true, y_pred):
    SS_res = K.sum(K.square(y_true - y_pred)) 
    SS_tot = K.sum(K.square(y_true - K.mean(y_true))) 
    return (1 - SS_res/(SS_tot + K.epsilon()))

model.compile(loss='mean_squared_error',
                    optimizer=optimizer,
              metrics=[r2_keras])
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 58, 300)           19349400  
_________________________________________________________________
lstm (LSTM)                  (None, 58, 128)           219648    
_________________________________________________________________
dropout (Dropout)            (None, 58, 128)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 129       
Total params: 19,700,761
Trainable params: 351,361
Non-trainable params: 19,349,400
______________________________________

#### Prepare train/validation sets

In [27]:
train_pct = 0.90

y = news_data.SENTIMENT.values
y_train, y_test = y[:int(data.shape[0] * train_pct)], \
                  y[-int(data.shape[0] * (1-train_pct)):]

X_train, X_test = data[:int(data.shape[0] * train_pct)], \
                  data[-int(data.shape[0] * (1-train_pct)):], \

print('train: {:,}\ntest: {:,}'.format(len(X_train), len(X_test)))

train: 502,692
test: 55,854


#### Fit the model

In [28]:
model.fit(X_train, y_train,
          epochs = 10,
          batch_size = 1024,
          shuffle=True,
          validation_split=0.2,
          validation_data=(X_test, y_test))

Train on 502692 samples, validate on 55854 samples
Epoch 1/10
 23552/502692 [>.............................] - ETA: 10:58 - loss: 0.3332 - r2_keras: -0.4449

KeyboardInterrupt: 

In [45]:
model.fit(X_train, y_train,
          epochs = 10,
          batch_size = 1024,
          shuffle=True,
          validation_split=0.2,
          validation_data=(X_test, y_test))


Train on 502692 samples, validate on 55854 samples
Epoch 1/10
502692/502692 [==============================] - 69s 138us/sample - loss: 0.2349 - r2_keras: -0.0109 - val_loss: 0.1081 - val_r2_keras: 0.5329
Epoch 2/10
502692/502692 [==============================] - 64s 127us/sample - loss: 0.0691 - r2_keras: 0.7025 - val_loss: 0.0506 - val_r2_keras: 0.7817
Epoch 3/10
502692/502692 [==============================] - 63s 126us/sample - loss: 0.0571 - r2_keras: 0.7542 - val_loss: 0.0454 - val_r2_keras: 0.8041
Epoch 4/10
502692/502692 [==============================] - 64s 128us/sample - loss: 0.0541 - r2_keras: 0.7671 - val_loss: 0.0443 - val_r2_keras: 0.8088
Epoch 5/10
502692/502692 [==============================] - 65s 129us/sample - loss: 0.0524 - r2_keras: 0.7744 - val_loss: 0.0426 - val_r2_keras: 0.8157
Epoch 6/10
502692/502692 [==============================] - 63s 126us/sample - loss: 0.0510 - r2_keras: 0.7804 - val_loss: 0.0424 - val_r2_keras: 0.8173
Epoch 7/10
502692/502692 [====

#### Save Model

In [ ]:
# model.save('model/tf-lstm-model.h5')
    
# with open('rp_model/tokenizer.pkl', 'wb') as f:
#     pickle.dump(tokenizer, f)

#### Load Model

In [29]:
model.load_weights('model/tf-lstm-model.h5')

Here we create a simple helper function to format text data into machine-readable format

In [30]:
def predict(txt):
    if isinstance(txt, str):
        txt = keras.preprocessing.sequence.pad_sequences(
            [encode(txt)],
            value=tokenizer.vocab["<PAD>"],
            padding='post',
            maxlen=maxlen)
    return model.predict(txt)[0][0]

In [31]:
predict('CEO found guilty of fraud')

-0.54968196

In [32]:
predict('IBM misses earnings expectations')

-0.6829254

In [33]:
predict('''Morgan Stanley analyst Andrew Berens initiates coverage
           on Blueprint Medicines (NASDAQ:BPMC) with a Overweight''')

0.93568856

In [34]:
predict('''Senex Energy Upgraded to Outperform from Neutral by Credit Suisse''')

0.9526496

In [38]:
predict('''Qbe Insurance Downgraded to Hold from Buy by Bell Potter''')

-0.58416826

### Sentiment and Topic Prediction model: Keras functional API

#### Build the model

<img src="img/lstm_bi_2o_diagram.png">

#### Load data

In [39]:
news_data = pd.read_csv('data/news_data.csv',
                        usecols = ['TEXT', 'SENTIMENT', 'TOPIC'])
news_data.drop_duplicates(inplace=True)

In [40]:
news_data.tail()

,SENTIMENT,TOPIC,TEXT
611430,0.94,analyst-ratings-change,Commonwealth Bank of Australia Upgraded to Buy from Hold by Bell Potter
611431,-0.58,analyst-ratings-change,Qbe Insurance Downgraded to Hold from Buy by Bell Potter
611432,0.49,partnership,Collective & Roland Team for
611433,-0.58,analyst-ratings-change,Mcmillan Shakespeare Downgraded to Neutral from Outperform by Macquarie
611434,0.95,analyst-ratings-set,Nearmap Initiated at Outperform by Macquarie


In [41]:
from sklearn.preprocessing import LabelBinarizer
encoder_topic = preprocessing.LabelBinarizer()
encoder_topic.fit(news_data.TOPIC.values)
print('# topics: {}\n'.format(len(encoder_topic.classes_)))

# topics: 220



In [45]:
neurons = 128
dropout = 0.20
topic_output_length = encoder_topic.classes_.shape[0]

my_embedding = layers.Embedding(input_dim=df_embedding.shape[0],
                                output_dim=df_embedding.shape[1],
                                input_length=maxlen,
                                trainable=False,
                                name='embedding_layer'
                               )
my_embedding.build((None, ))
my_embedding.set_weights([df_embedding.values])

sequence_input = keras.layers.Input((maxlen, ), name='sequence_input')

X = my_embedding(sequence_input)
X = layers.Bidirectional(
    layers.LSTM(units=neurons,
                return_sequences=True,
                dropout=dropout),
    name='bi_lstm_1')(X)
X = layers.LSTM(units=neurons,
                return_sequences=False,
                dropout=dropout,
                name='lstm_1')(X)

# Two output layers: softmax for Topic and 1D tanh for sentiment
topic_output = layers.Dense(topic_output_length,
                            activation='softmax',
                            name='topic')(X)
sentiment = layers.Dense(units=1,
                         activation='tanh',
                         name='sentiment')(X)

model = keras.models.Model(inputs=[sequence_input],
                           outputs=[topic_output, sentiment],
                           name='topic_sent_model')

optimizer = tf.optimizers.Adam(learning_rate=0.0001)

def r_squared(y_true, y_pred):
    SS_res =  K.sum(K.square(y_true - y_pred)) 
    SS_tot = K.sum(K.square(y_true - K.mean(y_true))) 
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

model.compile(loss={'topic':'categorical_crossentropy',
                    'sentiment':'mean_squared_error'},
                    optimizer=optimizer,
              metrics={'topic':'accuracy',
                       'sentiment':r_squared})

model.summary()

Model: "topic_sent_model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
sequence_input (InputLayer)     [(None, 58)]         0                                            
__________________________________________________________________________________________________
embedding_layer (Embedding)     (None, 58, 300)      19349400    sequence_input[0][0]             
__________________________________________________________________________________________________
bi_lstm_1 (Bidirectional)       (None, 58, 256)      439296      embedding_layer[0][0]            
__________________________________________________________________________________________________
lstm_1 (LSTM)                   (None, 128)          197120      bi_lstm_1[0][0]                  
___________________________________________________________________________________

#### Prepare train/validation sets

In [46]:
train_pct = 0.90
sent = news_data.SENTIMENT.values
topic = encoder_topic.transform(news_data.TOPIC.values)

X_train, X_test = data[:int(data.shape[0] * train_pct)], \
                  data[-int(data.shape[0] * (1-train_pct)):], \

topic_train, topic_test = topic[:int(data.shape[0] * train_pct)], \
                          topic[-int(data.shape[0] * (1-train_pct)):]

sent_train, sent_test = sent[:int(data.shape[0] * train_pct)], \
                        sent[-int(data.shape[0] * (1-train_pct)):]

print('train: {:,}\ntest: {:,}'.format(len(X_train), len(X_test)))

train: 502,692
test: 55,854


#### Fit the model

In [47]:
model.fit(X_train, [topic_train, sent_train],
          epochs = 10,
          batch_size = 1024,
          shuffle=True,
          validation_split=0.1,
          validation_data=(X_test, [topic_test, sent_test]))

W0806 18:09:48.327771 4793939392 deprecation.py:323] From /Users/mkangrga/anaconda3/envs/tf/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 502692 samples, validate on 55854 samples
Epoch 1/10
 26624/502692 [>.............................] - ETA: 19:21 - loss: 5.4235 - topic_loss: 5.1683 - sentiment_loss: 0.2552 - topic_accuracy: 0.0556 - sentiment_r_squared: -0.0954

KeyboardInterrupt: 

#### Load model

In [48]:
model.load_weights('model/tf-lstm-model-topic-sent.h5')
with open('model/encoder_topic.pkl', 'rb') as f:
    encoder_topic = pickle.load(f)
    
# model.save('model/tf-lstm-model-topic-sent.h5')
# with open('model/encoder_topic.pkl', 'wb') as f:
#     pickle.dump(encoder_topic, f)

In [49]:
def predict(txt):
    if isinstance(txt, str):
        txt = keras.preprocessing.sequence.pad_sequences(
            [encode(txt)],
            value=tokenizer.vocab["<PAD>"],
            padding='post',
            maxlen=maxlen)
        predictions = model.predict(txt)
        topic = encoder_topic.inverse_transform(predictions[0])[0]
        sentiment = predictions[1][0][0]
    return topic, sentiment

In [50]:
predict('CEO found guilty of fraud')

('executive-resignation', -0.6195352)

In [51]:
predict('IBM misses revenue expectations')

('revenue', -0.26360473)

In [52]:
predict('''Morgan Stanley analyst Andrew Berens initiates coverage
           on Blueprint Medicines (NASDAQ:BPMC) with a Overweight''')

('analyst-ratings-set', 0.9304361)

In [53]:
predict('''Senex Energy Upgraded to Outperform from Neutral by Credit Suisse''')

('analyst-ratings-change', 0.7553701)

In [54]:
predict('''Qbe Insurance Downgraded to Hold from Buy by Bell Potter''')

('analyst-ratings-change', -0.6919683)